# Земских Денис
## Hard-способ PyTorch ResNet50

In [2]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [3]:
# Импорт необходимых библиотек
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import timm
import numpy as np
from PIL import Image
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import os
from roboflow import Roboflow
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
from tqdm import tqdm

class Config:
    ROBOFLOW_API_KEY = "2UgAsIW11EHQmt8gh36L"
    WORKSPACE = "wildfire-kwxfu"
    PROJECT = "wildfire-tksrf"
    VERSION = 4
    
    BATCH_SIZE = 4
    NUM_EPOCHS = 20
    LEARNING_RATE = 0.005
    MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0005
    STEP_SIZE = 7
    GAMMA = 0.1
    NUM_CLASSES = 3
    
    DATA_DIR = "/kaggle/working/"
    CHECKPOINT_DIR = "./checkpoints"
    PLOTS_DIR = "./plots"
    EXAMPLES_DIR = "./examples"

def download_dataset():
    rf = Roboflow(api_key=Config.ROBOFLOW_API_KEY)
    project = rf.workspace(Config.WORKSPACE).project(Config.PROJECT)
    dataset = project.version(Config.VERSION).download("coco")
    return dataset

class SatelliteDataset(Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))
        
    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        annotations = coco.loadAnns(ann_ids)
        
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.root, img_info['file_name'])
        img = Image.open(img_path).convert("RGB")
        
        boxes = []
        labels = []
        for ann in annotations:
            xmin, ymin, width, height = ann['bbox']
            xmax = xmin + width
            ymax = ymin + height
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(ann['category_id'])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([img_id])
        }
        
        if self.transforms:
            img = self.transforms(img)
            
        return img, target
    
    def __len__(self):
        return len(self.ids)

# Создание модели с кастомным backbone
def create_model():
    backbone = torchvision.models.detection.backbone_utils.resnet_fpn_backbone(
        backbone_name='resnet50', 
        pretrained=True
    )
    
    anchor_sizes = ((32,), (64,), (128,), (256,), (512,))
    aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
    anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)
    
    model = FasterRCNN(
        backbone,
        num_classes=Config.NUM_CLASSES,
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=torchvision.ops.MultiScaleRoIAlign(
            featmap_names=['0', '1', '2', '3'],
            output_size=7,
            sampling_ratio=2
        )
    )
    return model

def collate_fn(batch):
    return tuple(zip(*batch))

# Визуализация предсказаний
def visualize_predictions(model, dataset, device, save_dir, num_examples=3):
    model.eval()
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    for i in range(num_examples):
        img, target = dataset[np.random.randint(0, len(dataset))]
        with torch.no_grad():
            prediction = model([img.to(device)])[0]
        
        fig, ax = plt.subplots(1, figsize=(12, 8))
        img = img.cpu().permute(1, 2, 0).numpy()
        ax.imshow(img)
        
        for box, label in zip(target['boxes'], target['labels']):
            xmin, ymin, xmax, ymax = box
            rect = patches.Rectangle(
                (xmin, ymin), xmax-xmin, ymax-ymin,
                linewidth=1, edgecolor='g', facecolor='none'
            )
            ax.add_patch(rect)
            ax.text(xmin, ymin, f'GT: {label}', color='g')
        
        for box, label, score in zip(
            prediction['boxes'], prediction['labels'], prediction['scores']
        ):
            if score > 0.5:
                xmin, ymin, xmax, ymax = box.cpu().numpy()
                rect = patches.Rectangle(
                    (xmin, ymin), xmax-xmin, ymax-ymin,
                    linewidth=1, edgecolor='r', facecolor='none'
                )
                ax.add_patch(rect)
                ax.text(xmin, ymin, f'Pred: {label} ({score:.2f})', color='r')
        
        plt.savefig(os.path.join(save_dir, f'example_{i}.png'))
        plt.close()

def evaluate_coco(model, dataset, device, save_dir):
    model.eval()
    results = []
    coco = dataset.coco
    
    for idx in tqdm(range(len(dataset))):
        img, target = dataset[idx]
        img = img.to(device)
        
        with torch.no_grad():
            prediction = model([img])[0]
        
        image_id = target['image_id'].item()
        
        for box, label, score in zip(
            prediction['boxes'], prediction['labels'], prediction['scores']
        ):
            box = box.cpu().numpy().tolist()
            score = score.cpu().item()
            label = label.cpu().item()
            
            results.append({
                "image_id": image_id,
                "category_id": label,
                "bbox": [box[0], box[1], box[2]-box[0], box[3]-box[1]],
                "score": score
            })
    
    results_path = os.path.join(save_dir, "predictions.json")
    with open(results_path, 'w') as f:
        json.dump(results, f)
    
    coco_pred = coco.loadRes(results_path)
    coco_eval = COCOeval(coco, coco_pred, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    
    return coco_eval.stats

def plot_training_metrics(train_losses, val_losses, map_scores, save_dir):
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(2, 1, 2)
    plt.plot(map_scores, label='mAP@0.5:0.95', color='green')
    plt.title('Validation mAP')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'training_metrics.png'))
    plt.close()

torch.cuda.empty_cache()
os.makedirs(Config.DATA_DIR, exist_ok=True)
os.makedirs(Config.CHECKPOINT_DIR, exist_ok=True)

os.makedirs(Config.PLOTS_DIR, exist_ok=True)
os.makedirs(Config.EXAMPLES_DIR, exist_ok=True)
dataset = download_dataset()

train_dataset = SatelliteDataset(
    os.path.join(dataset.location, "train"),
    os.path.join(dataset.location, "train", "_annotations.coco.json"),
    F.to_tensor
)

val_dataset = SatelliteDataset(
    os.path.join(dataset.location, "valid"),
    os.path.join(dataset.location, "valid", "_annotations.coco.json"),
    F.to_tensor
)

test_dataset = SatelliteDataset(
    os.path.join(dataset.location, "test"),
    os.path.join(dataset.location, "test", "_annotations.coco.json"),
    F.to_tensor
)

train_loader = DataLoader(
    train_dataset,
    batch_size=Config.BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE, collate_fn=collate_fn)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = create_model().to(device)

optimizer = optim.SGD(
    model.parameters(),
    lr=Config.LEARNING_RATE,
    momentum=Config.MOMENTUM,
    weight_decay=Config.WEIGHT_DECAY
)
scheduler = StepLR(optimizer, step_size=Config.STEP_SIZE, gamma=Config.GAMMA)

train_losses = []
val_losses = []
map_scores = []

for epoch in tqdm(range(Config.NUM_EPOCHS)):
    print("Фаза обучения")
    model.train()
    epoch_train_loss = 0
    for images, targets in tqdm(train_loader, desc=f'Epoch {epoch} Training'):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        epoch_train_loss += losses.item()
    
    train_losses.append(epoch_train_loss / len(train_loader))
    
    print("Фаза валидации")
    model.eval()
    epoch_val_loss = 0
    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc=f'Epoch {epoch} Validation'):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            epoch_val_loss += losses.item()
    
    val_losses.append(epoch_val_loss / len(val_loader))
    model.eval()
    
    print("Оценка mAP на валидации")
    coco_stats = evaluate_coco(model, val_dataset, device, Config.PLOTS_DIR)
    map_scores.append(coco_stats[0])
    
    print("Обновление шедулера")
    scheduler.step()
    
    print("Сохранение модели")
    torch.save(
        model.state_dict(),
        os.path.join(Config.CHECKPOINT_DIR, f"epoch_{epoch}_map_{coco_stats[0]:.3f}.pth")
    )
    
    print(f"Epoch {epoch} | "
            f"Train Loss: {train_losses[-1]:.3f} | "
            f"Val Loss: {val_losses[-1]:.3f} | "
            f"mAP@0.5:0.95: {map_scores[-1]:.3f}")

plot_training_metrics(train_losses, val_losses, map_scores, Config.PLOTS_DIR)

print("\nEvaluating on test set...")
test_stats = evaluate_coco(model, test_dataset, device, Config.PLOTS_DIR)
print(f"Тест mAP@0.5:0.95: {test_stats[0]:.3f}")

visualize_predictions(model, test_dataset, device, Config.EXAMPLES_DIR)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Wildfire-4 in coco:: 100%|██████████| 3884/3884 [00:00<00:00, 7894.14it/s]
The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 206MB/s]
  0%|          | 0/20 [00:00<?, ?it/s]

Фаза обучения


...

100%|██████████| 20/20 [2:06:28<00:00, 379.44s/it]

Epoch 19 | Train Loss: 0.243 | Val Loss: 0.458 | mAP@0.5:0.95: 0.195



Evaluating on test set...


100%|██████████| 276/276 [00:14<00:00, 18.90it/s]


Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.615
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.227
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.286
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.048
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.149
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.349
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.365
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=10